# 필요 모듈

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd


# 데이터 불러오기

In [3]:
df_loan = pd.read_csv('/content/drive/MyDrive/loan_result.csv')
df_log = pd.read_csv('/content/drive/MyDrive/log_data.csv')
df_user = pd.read_csv('/content/drive/MyDrive/user_spec.csv')

## 결측치 확인

In [4]:
df_loan_sum = df_loan.isnull().sum()/len(df_loan)*100
df_log_sum = df_log.isnull().sum()/len(df_log)*100
df_user_sum = df_user.isnull().sum()/len(df_user)*100

In [5]:
df_loan_sum

application_id            0.000000
loanapply_insert_time     0.000000
bank_id                   0.000000
product_id                0.000000
loan_limit                0.055406
loan_rate                 0.055406
is_applied               24.078891
dtype: float64

In [6]:
df_log_sum

user_id           0.000000
event             0.000000
timestamp         0.000000
mp_os             0.005492
mp_app_version    3.702069
date_cd           0.000000
dtype: float64

In [7]:
df_user_sum

application_id                          0.000000
user_id                                 0.000000
birth_year                              0.929626
gender                                  0.929626
insert_time                             0.000000
credit_score                            7.539363
yearly_income                           0.006455
income_type                             0.006097
company_enter_month                    12.319468
employment_type                         0.006097
houseown_type                           0.006097
desired_amount                          0.006097
purpose                                 0.006097
personal_rehabilitation_yn             42.135580
personal_rehabilitation_complete_yn    86.310443
existing_loan_cnt                      14.241409
existing_loan_amt                      22.505408
dtype: float64

## 데이터분포 확인 및 전처리

### 영어를 한글로 통일

In [8]:
df_user['purpose'].value_counts()

생활비             866079
대환대출            353794
사업자금             62115
기타               30132
전월세보증금           27236
주택구입             17471
LIVING           15432
투자               12532
자동차구입             3394
SWITCHLOAN        3248
BUSINESS           935
ETC                593
HOUSEDEPOSIT       562
BUYHOUSE           307
INVEST             166
BUYCAR             135
Name: purpose, dtype: int64

In [9]:
df_user.loc[df_user['purpose']=='BUYCAR','purpose'] = '자동차구입'
df_user.loc[df_user['purpose']=='BUYHOUSE','purpose'] = '주택구입'
df_user.loc[df_user['purpose']=='HOUSEDEPOSIT', 'purpose'] = '전월세보증금'
df_user.loc[df_user['purpose']=='SWITCHLOAN', 'purpose'] = '대환대출'
df_user.loc[df_user['purpose']=='INVEST', 'purpose'] = '투자'
df_user.loc[df_user['purpose']=='LIVING', 'purpose'] = '생활비'
df_user.loc[df_user['purpose']=='BUSINESS', 'purpose'] = '사업자금'
df_user.loc[df_user['purpose']=='ETC', 'purpose'] = '기타'

In [10]:
df_user['purpose'].value_counts()

생활비       881511
대환대출      357042
사업자금       63050
기타         30725
전월세보증금     27798
주택구입       17778
투자         12698
자동차구입       3529
Name: purpose, dtype: int64

### 각 데이터 user_id, application_id 기준으로 중복되지 않은 행 삭제

In [11]:
df_app = df_loan['application_id']
df_app = pd.DataFrame(df_app)
df_app = df_app.drop_duplicates(subset = None, keep = 'first', inplace = False, ignore_index = False)

In [12]:
df_user_id = df_user['user_id']
df_user_id = pd.DataFrame(df_user_id)
df_user_id = df_user_id.drop_duplicates(subset = None, keep = 'first', inplace = False, ignore_index = False)

In [13]:
df_user = pd.merge(df_app, df_user)

In [14]:
df_user = pd.merge(df_user_id, df_user)

In [15]:
df_user

,user_id,application_id,birth_year,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt
0,118218,814553,1985.0,1.0,2022-06-23 14:40:44,680.0,108000000.0,PRIVATEBUSINESS,20151102.0,기타,자가,1000000.0,생활비,0.0,NaN,4.0,162000000.0
1,118218,2158388,1985.0,1.0,2022-03-04 15:58:19,690.0,100000000.0,PRIVATEBUSINESS,201511.0,기타,자가,10000000.0,사업자금,NaN,NaN,4.0,162000000.0
2,118218,658694,1985.0,1.0,2022-05-31 10:54:41,660.0,108000000.0,PRIVATEBUSINESS,201511.0,기타,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0
3,118218,193061,1985.0,1.0,2022-05-16 13:33:55,660.0,100000000.0,PRIVATEBUSINESS,201511.0,계약직,자가,1000000.0,사업자금,0.0,NaN,4.0,162000000.0
4,118218,737407,1985.0,1.0,2022-06-02 12:33:28,660.0,105000000.0,PRIVATEBUSINESS,201511.0,계약직,자가,1000000.0,기타,0.0,NaN,4.0,162000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968748,53701,316356,1996.0,1.0,2022-03-22 10:01:38,730.0,60000000.0,PRIVATEBUSINESS,202105.0,정규직,전월세,15000000.0,사업자금,NaN,NaN,1.0,NaN
968749,489900,1864587,2000.0,1.0,2022-03-22 14:55:32,590.0,25000000.0,FREELANCER,202106.0,기타,기타가족소유,5000000.0,사업자금,NaN,NaN,NaN,NaN
968750,151422,1327066,1955.0,1.0,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,NaN,기타,자가,50000000.0,생활비,NaN,NaN,1.0,NaN
968751,173524,1319606,1983.0,1.0,2022-03-22 07:34:32,750.0,75000000.0,EARNEDINCOME,200908.0,정규직,자가,100000000.0,대환대출,NaN,NaN,8.0,200000000.0


In [16]:
df_user_id1 = df_user['user_id']
df_user_id1 = pd.DataFrame(df_user_id1)
df_user_id1 = df_user_id1.drop_duplicates(subset = None, keep = 'first', inplace = False, ignore_index = False)

In [17]:
df_user_id1

,user_id
0,118218
6,553686
10,59516
12,167320
19,33400
...,...
968748,53701
968749,489900
968750,151422
968751,173524


In [18]:
df_app1 = df_user['application_id']
df_app1 = pd.DataFrame(df_app1)
df_app1 = df_app1.drop_duplicates(subset = None, keep = 'first', inplace = False, ignore_index = False)

In [19]:
df_log = pd.merge(df_user_id1, df_log)
df_loan = pd.merge(df_app1, df_loan)

In [20]:
df_loan

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,814553,2022-06-23 14:40:45,13,123,0.0,18.6,NaN
1,2158388,2022-03-04 15:58:21,44,171,49000000.0,14.4,0.0
2,658694,2022-05-31 10:54:42,30,85,20000000.0,17.2,0.0
3,658694,2022-05-31 10:54:42,19,231,5000000.0,16.6,0.0
4,193061,2022-05-16 13:33:56,30,85,20000000.0,17.2,0.0
...,...,...,...,...,...,...,...
13527245,816537,2022-03-22 08:55:14,13,123,8000000.0,20.0,0.0
13527246,816537,2022-03-22 08:55:14,19,231,50000000.0,18.9,0.0
13527247,816537,2022-03-22 08:55:15,6,36,16000000.0,14.2,0.0
13527248,816537,2022-03-22 08:55:15,42,258,4000000.0,12.9,0.0


### 필요없는 컬럼 삭제

In [21]:
df_log

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,118218,GetCreditInfo,2022-03-02 19:44:40,android,447,2022-03-02
1,118218,GetCreditInfo,2022-06-23 14:39:20,android,469,2022-06-23
2,118218,GetCreditInfo,2022-04-18 11:04:06,android,460,2022-04-18
3,118218,GetCreditInfo,2022-03-28 11:09:28,android,451,2022-03-28
4,553686,Login,2022-05-08 14:47:56,Android,3.12.1,2022-05-08
...,...,...,...,...,...,...
14344017,173524,EndLoanApply,2022-03-22 07:35:59,iOS,3.6.1,2022-03-22
14344018,173524,GetCreditInfo,2022-03-25 08:41:33,iOS,3.6.1,2022-03-25
14344019,173524,GetCreditInfo,2022-03-22 07:30:18,iOS,3.6.1,2022-03-22
14344020,173524,GetCreditInfo,2022-03-22 07:31:33,iOS,3.6.1,2022-03-22


In [22]:
df_log.isnull().sum()

user_id                0
event                  0
timestamp              0
mp_os                377
mp_app_version    550049
date_cd                0
dtype: int64

#### mp_app_version의 null 값이 크고 버전에 따라 새롭게 추가되는 기능들이 있지만 분석 결과에 크게 영향을 미치지 않는다 생각하여 삭제함
#### mp_os는 mp_app_version을 확인할때 필요한 컬럼이지만, app_version 컬럼을 삭제하므로 마찬가지로 제거한다,

In [23]:
df_log = df_log.drop(['mp_os','mp_app_version'], axis = 1)

#### event에서 openapp, endloanapply, login, getcreditinfo, startloanapplu, viewloanapplyintro,signup은 직접적인 하나로 연결되는 부분이기에 제거

In [24]:
li = ['UseLoanManage','UsePrepayCalc','UseDSRCalc','GetCreditInfo']
df_log = df_log[df_log['event'].isin(li)]
df_log

,user_id,event,timestamp,date_cd
0,118218,GetCreditInfo,2022-03-02 19:44:40,2022-03-02
1,118218,GetCreditInfo,2022-06-23 14:39:20,2022-06-23
2,118218,GetCreditInfo,2022-04-18 11:04:06,2022-04-18
3,118218,GetCreditInfo,2022-03-28 11:09:28,2022-03-28
19,553686,UseLoanManage,2022-05-19 13:52:04,2022-05-19
...,...,...,...,...
14344013,173524,UseLoanManage,2022-03-22 07:32:27,2022-03-22
14344018,173524,GetCreditInfo,2022-03-25 08:41:33,2022-03-25
14344019,173524,GetCreditInfo,2022-03-22 07:30:18,2022-03-22
14344020,173524,GetCreditInfo,2022-03-22 07:31:33,2022-03-22


### 나이와 성별 컬럼 비교

In [25]:
df_user['Old'] = 2022 - df_user['birth_year']

In [26]:
# 20대
df_m20 =  df_user[(df_user['Old'] < 30) & (df_user['gender']==1.0)]
df_w20 =  df_user[(df_user['Old'] < 30) & (df_user['gender']==0.0)]

# 30대
df_m30 =  df_user[(df_user['Old'] >= 30) & (df_user['Old'] < 40) & (df_user['gender']==1.0)]
df_w30 =  df_user[(df_user['Old'] >= 30) & (df_user['Old'] < 40) & (df_user['gender']==0.0)]

# 40대
df_m40 =  df_user[(df_user['Old'] >= 40) & (df_user['Old'] < 50) & (df_user['gender']==1.0)]
df_w40 =  df_user[(df_user['Old'] >= 40) & (df_user['Old'] < 50) & (df_user['gender']==0.0)]

# 50대
df_m50 =  df_user[(df_user['Old'] >= 50) & (df_user['Old'] < 60) & (df_user['gender']==1.0)]
df_w50 =  df_user[(df_user['Old'] >= 50) & (df_user['Old'] < 60) & (df_user['gender']==0.0)]

In [27]:
# 20대
df_m20_sum = df_m20['purpose'].value_counts()/len(df_m20)*100
df_w20_sum = df_w20['purpose'].value_counts()/len(df_w20)*100

# 30대
df_m30_sum = df_m30['purpose'].value_counts()/len(df_m30)*100
df_w30_sum = df_w30['purpose'].value_counts()/len(df_w30)*100

# 40대
df_m40_sum = df_m40['purpose'].value_counts()/len(df_m40)*100
df_w40_sum = df_w40['purpose'].value_counts()/len(df_w40)*100

# 50대
df_m50_sum = df_m50['purpose'].value_counts()/len(df_m50)*100
df_w50_sum = df_w50['purpose'].value_counts()/len(df_w50)*100

In [28]:
df_m20_sum

생활비       63.964088
대환대출      23.473836
사업자금       3.294852
전월세보증금     2.944982
기타         2.417285
투자         1.999465
주택구입       1.307676
자동차구입      0.597815
Name: purpose, dtype: float64

In [29]:
df_w20_sum

생활비       60.581867
대환대출      26.075778
전월세보증금     5.099797
사업자금       3.602842
기타         2.242896
주택구입       1.657645
투자         0.520974
자동차구입      0.218200
Name: purpose, dtype: float64

In [30]:
df_m50_sum

생활비       61.353970
대환대출      25.885077
사업자금       6.102005
기타         2.718908
전월세보증금     1.529462
주택구입       1.280848
투자         0.933520
자동차구입      0.196210
Name: purpose, dtype: float64

In [31]:
df_w50_sum

생활비       61.640932
대환대출      25.289364
사업자금       5.787280
기타         3.036866
전월세보증금     1.925988
주택구입       1.665153
투자         0.500710
자동차구입      0.153706
Name: purpose, dtype: float64

In [32]:
df_user = df_user.drop(['birth_year'], axis = 1)

### 개인회생자, 기대출 컬럼의 null값은 기대출을 받지 않고, 개인회생자가 아닌 경우로 여겨 0으로 채움

In [33]:
df_user.loc[df_user['personal_rehabilitation_yn'] != df_user['personal_rehabilitation_yn'], 'personal_rehabilitation_yn'] = 0
df_user.loc[df_user['personal_rehabilitation_complete_yn'] != df_user['personal_rehabilitation_complete_yn'], 'personal_rehabilitation_complete_yn'] = 0
df_user.loc[df_user['existing_loan_cnt'] != df_user['existing_loan_cnt'], 'existing_loan_cnt'] = 0
df_user.loc[df_user['existing_loan_amt'] != df_user['existing_loan_amt'], 'existing_loan_amt'] = 0

### 개인회생자가 아닌데 납입완료로 표시된 경우 제거

In [34]:
# 개인회생자 이면서 납입 완료
df_user[(df_user['personal_rehabilitation_yn']==1.0) & (df_user['personal_rehabilitation_complete_yn']==1.0)]

,user_id,application_id,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,Old
10685,340624,70776,1.0,2022-05-29 09:36:49,730.0,35000000.0,EARNEDINCOME,201806.0,정규직,전월세,50000000.0,생활비,1.0,1.0,4.0,90000000.0,56.0
12578,675357,1960696,0.0,2022-05-08 09:23:39,620.0,28000000.0,EARNEDINCOME,202204.0,정규직,전월세,5000000.0,생활비,1.0,1.0,4.0,33000000.0,47.0
12579,675357,332122,0.0,2022-05-05 19:41:29,620.0,24000000.0,EARNEDINCOME,202107.0,정규직,기타가족소유,30000000.0,대환대출,1.0,1.0,4.0,33000000.0,47.0
12580,675357,1992918,0.0,2022-05-15 18:14:41,620.0,28000000.0,EARNEDINCOME,202204.0,정규직,전월세,20000000.0,대환대출,1.0,1.0,4.0,33000000.0,47.0
12582,675357,815168,0.0,2022-05-06 12:46:58,620.0,24000000.0,EARNEDINCOME2,202107.0,정규직,전월세,20000000.0,대환대출,1.0,1.0,4.0,33000000.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963111,632459,1372671,1.0,2022-06-24 12:58:48,630.0,30000000.0,EARNEDINCOME2,20190624.0,정규직,전월세,3000000.0,생활비,1.0,1.0,0.0,0.0,35.0
963123,383709,1100780,1.0,2022-06-26 12:54:34,930.0,20000000.0,EARNEDINCOME,20171123.0,기타,기타가족소유,10000000.0,전월세보증금,1.0,1.0,1.0,0.0,51.0
963420,291777,947142,1.0,2022-04-27 16:02:12,780.0,30000000.0,EARNEDINCOME,200101.0,정규직,자가,20000000.0,생활비,1.0,1.0,2.0,11000000.0,54.0
964930,61361,2137105,1.0,2022-05-18 17:45:23,740.0,100000000.0,PRIVATEBUSINESS,202001.0,기타,기타가족소유,70000000.0,대환대출,1.0,1.0,3.0,71000000.0,42.0


In [35]:
# 개인회생자가 아닌데 납입 완료인 경우
df_user[(df_user['personal_rehabilitation_yn']==0.0) & (df_user['personal_rehabilitation_complete_yn']==1.0)]

,user_id,application_id,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,Old
330848,24030,1624053,1.0,2022-05-13 00:23:08,650.0,32000000.0,EARNEDINCOME,202204.0,정규직,기타가족소유,5000000.0,생활비,0.0,1.0,1.0,3000000.0,41.0
362976,39991,1574646,0.0,2022-04-21 13:14:59,600.0,22000000.0,EARNEDINCOME,201703.0,계약직,전월세,4000000.0,생활비,0.0,1.0,1.0,0.0,41.0
463588,598997,1065260,1.0,2022-05-17 19:42:47,NaN,15000000.0,PRIVATEBUSINESS,201706.0,기타,전월세,10000000.0,생활비,0.0,1.0,0.0,0.0,34.0


In [36]:
index1 = df_user[(df_user['personal_rehabilitation_yn']==0.0) & (df_user['personal_rehabilitation_complete_yn']==1.0)].index
df_user=df_user.drop(index1)

### loan_limit, loan_rate 결측치 있는 행 삭제

In [37]:
df_loan

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,814553,2022-06-23 14:40:45,13,123,0.0,18.6,NaN
1,2158388,2022-03-04 15:58:21,44,171,49000000.0,14.4,0.0
2,658694,2022-05-31 10:54:42,30,85,20000000.0,17.2,0.0
3,658694,2022-05-31 10:54:42,19,231,5000000.0,16.6,0.0
4,193061,2022-05-16 13:33:56,30,85,20000000.0,17.2,0.0
...,...,...,...,...,...,...,...
13527245,816537,2022-03-22 08:55:14,13,123,8000000.0,20.0,0.0
13527246,816537,2022-03-22 08:55:14,19,231,50000000.0,18.9,0.0
13527247,816537,2022-03-22 08:55:15,6,36,16000000.0,14.2,0.0
13527248,816537,2022-03-22 08:55:15,42,258,4000000.0,12.9,0.0


In [38]:
df_loan = df_loan.dropna(subset=['loan_limit','loan_rate'])

### loan_limit가 0인 경우와 10억을 넘어가는 경우

In [39]:
df_loan[(df_loan['loan_limit'] == 0) & (df_loan['is_applied'] == 1)]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
49298,192407,2022-04-04 20:14:28,5,194,0.0,14.9,1.0
122790,1526007,2022-04-08 20:38:29,14,197,0.0,19.9,1.0
130432,1445614,2022-04-15 04:51:41,12,35,0.0,19.9,1.0
173227,1069779,2022-03-08 13:56:41,14,197,0.0,18.8,1.0
275640,2018493,2022-03-21 03:46:14,20,111,0.0,14.6,1.0
...,...,...,...,...,...,...,...
12944193,575406,2022-04-05 11:25:33,14,197,0.0,14.2,1.0
13012211,1651069,2022-03-16 10:23:07,14,197,0.0,19.9,1.0
13255428,942099,2022-05-20 14:09:35,14,197,0.0,19.9,1.0
13308881,1578562,2022-04-28 14:36:41,14,197,0.0,19.9,1.0


In [40]:
df_user[df_user['user_id'] ==395166]

,user_id,application_id,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,Old
5226,395166,330775,1.0,2022-04-16 19:15:49,590.0,50000000.0,EARNEDINCOME,202109.0,정규직,자가,3000000.0,생활비,0.0,0.0,5.0,87000000.0,43.0
5227,395166,192407,1.0,2022-04-04 20:14:27,590.0,50000000.0,EARNEDINCOME,202109.0,정규직,자가,5000000.0,생활비,0.0,0.0,5.0,87000000.0,43.0
5228,395166,1291056,1.0,2022-06-13 19:18:48,570.0,50000000.0,EARNEDINCOME,20210901.0,정규직,자가,3000000.0,생활비,0.0,0.0,5.0,87000000.0,43.0
5229,395166,1020633,1.0,2022-03-19 14:38:23,590.0,50000000.0,EARNEDINCOME,202109.0,정규직,자가,5000000.0,생활비,0.0,0.0,5.0,87000000.0,43.0
5230,395166,1447793,1.0,2022-06-21 22:41:56,570.0,50000000.0,EARNEDINCOME,20210901.0,정규직,자가,5000000.0,생활비,0.0,0.0,5.0,87000000.0,43.0
5231,395166,1443164,1.0,2022-06-16 10:40:49,570.0,50000000.0,EARNEDINCOME,20210901.0,정규직,자가,50000000.0,대환대출,0.0,0.0,5.0,87000000.0,43.0
5232,395166,584117,1.0,2022-03-28 20:57:46,590.0,50000000.0,EARNEDINCOME,202109.0,정규직,자가,5000000.0,생활비,0.0,0.0,5.0,87000000.0,43.0
5233,395166,1870985,1.0,2022-05-02 20:50:50,590.0,50000000.0,EARNEDINCOME,202109.0,정규직,자가,3000000.0,생활비,0.0,0.0,5.0,87000000.0,43.0


In [41]:
df_loan[df_loan['loan_limit'] > 1000000000] 

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
447265,1936078,2022-03-29 07:36:08,31,244,1.034000e+09,7.7,0.0
557255,1204858,2022-06-07 11:30:53,59,150,1.047000e+11,17.7,NaN
591580,1934129,2022-04-04 23:50:05,43,246,1.000000e+10,9.9,1.0
591595,912866,2022-06-08 08:24:19,43,246,1.000000e+10,12.9,NaN
591635,1687312,2022-03-29 09:36:30,43,246,1.000000e+10,9.9,0.0
591654,503070,2022-04-05 00:12:23,43,246,1.000000e+10,9.9,1.0
989003,905110,2022-03-29 01:01:24,31,244,1.208000e+09,7.7,0.0
1217306,639784,2022-03-01 14:33:29,43,246,1.000000e+10,9.9,1.0
1218359,850504,2022-03-02 21:00:48,43,246,1.000000e+10,9.9,1.0
1318453,1906479,2022-03-28 13:40:05,31,244,1.659000e+09,7.2,0.0


## user_data와 loan_data merge

In [42]:
df_loan['loanapply_insert_time']=pd.to_datetime(df_loan['loanapply_insert_time'])
df_log['timestamp']=pd.to_datetime(df_log['timestamp'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
df_loan['Month'] = df_loan['loanapply_insert_time'].dt.month
df_log['Month'] = df_log['timestamp'].dt.month

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
df_user_loan = pd.merge(df_user, df_loan)

### log_data event에 대하여 월별로 카운트

In [45]:
df_log_count = df_log.groupby(['user_id','Month'])['event'].count().reset_index()
df_user_loan = pd.merge(df_user_loan,df_log_count, how='left', on=['user_id','Month'], sort=False)
df_user_loan['event'] = df_user_loan['event'].fillna(0)

#### 한번 신청할때 application_id 개수 파악
#### 한 유저에 대한 3~6월까지의 신청횟수

In [46]:
df_user_loan

,user_id,application_id,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,...,existing_loan_amt,Old,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,Month,event
0,118218,814553,1.0,2022-06-23 14:40:44,680.0,108000000.0,PRIVATEBUSINESS,20151102.0,기타,자가,...,162000000.0,37.0,2022-06-23 14:40:45,13,123,0.0,18.6,NaN,6,1.0
1,118218,2158388,1.0,2022-03-04 15:58:19,690.0,100000000.0,PRIVATEBUSINESS,201511.0,기타,자가,...,162000000.0,37.0,2022-03-04 15:58:21,44,171,49000000.0,14.4,0.0,3,2.0
2,118218,658694,1.0,2022-05-31 10:54:41,660.0,108000000.0,PRIVATEBUSINESS,201511.0,기타,자가,...,162000000.0,37.0,2022-05-31 10:54:42,30,85,20000000.0,17.2,0.0,5,0.0
3,118218,658694,1.0,2022-05-31 10:54:41,660.0,108000000.0,PRIVATEBUSINESS,201511.0,기타,자가,...,162000000.0,37.0,2022-05-31 10:54:42,19,231,5000000.0,16.6,0.0,5,0.0
4,118218,193061,1.0,2022-05-16 13:33:55,660.0,100000000.0,PRIVATEBUSINESS,201511.0,계약직,자가,...,162000000.0,37.0,2022-05-16 13:33:56,30,85,20000000.0,17.2,0.0,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13519847,3864,816537,0.0,2022-03-22 08:55:14,NaN,35000000.0,FREELANCER,201103.0,기타,자가,...,0.0,45.0,2022-03-22 08:55:14,13,123,8000000.0,20.0,0.0,3,0.0
13519848,3864,816537,0.0,2022-03-22 08:55:14,NaN,35000000.0,FREELANCER,201103.0,기타,자가,...,0.0,45.0,2022-03-22 08:55:14,19,231,50000000.0,18.9,0.0,3,0.0
13519849,3864,816537,0.0,2022-03-22 08:55:14,NaN,35000000.0,FREELANCER,201103.0,기타,자가,...,0.0,45.0,2022-03-22 08:55:15,6,36,16000000.0,14.2,0.0,3,0.0
13519850,3864,816537,0.0,2022-03-22 08:55:14,NaN,35000000.0,FREELANCER,201103.0,기타,자가,...,0.0,45.0,2022-03-22 08:55:15,42,258,4000000.0,12.9,0.0,3,0.0


In [47]:
df = df_user_loan

In [48]:
df = df.drop_duplicates(['application_id','product_id'], keep = False)

In [49]:
df['application_id_1'] = df['application_id']
df_user['user_id_1'] = df_user['user_id']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
df_id_count = df.groupby(['user_id','application_id'])['application_id_1'].count().reset_index()
df1 = pd.merge(df, df_id_count, how='left', on=['user_id','application_id'], sort=False)

In [51]:
df_id_count_1 = df_user.groupby(['user_id'])['user_id_1'].count().reset_index()
df1= pd.merge(df_id_count_1, df1) 

In [52]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13518964 entries, 0 to 13518963
Data columns (total 28 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   user_id                              int64         
 1   user_id_1                            int64         
 2   application_id                       int64         
 3   gender                               float64       
 4   insert_time                          object        
 5   credit_score                         float64       
 6   yearly_income                        float64       
 7   income_type                          object        
 8   company_enter_month                  float64       
 9   employment_type                      object        
 10  houseown_type                        object        
 11  desired_amount                       float64       
 12  purpose                              object        
 13  personal_rehabilitation_y

## 단계별 선택법

### 단계별 선택법 데이터 이름 바꾸기

In [ ]:
import statsmodels.api as sm

In [ ]:
## 전진 단계별 선택법
variables = variables = df_train.drop(['is_applied'], axis = 1)  ## 설명 변수 리스트
 
y = df_train['is_applied']
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05
sl_remove = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        X = df_train[selected_variables+[col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = df_train[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(df_train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [ ]:
selected_variables

In [ ]:
df_encoding = df_encoding.drop(['purpose_기타','purpose_투자','purpose_주택구입','employment_type_기타','employment_type_일용직'], axis = 1)

### test set의 행은 제거 할수 없기에 mice를 이용하여 보간법을 시행
### train set의 null값은 제거

In [53]:
df1 = df1.drop(['user_id','application_id','application_id_1_x','loanapply_insert_time','insert_time','company_enter_month'], axis = 1)

In [54]:
df_encoding = pd.get_dummies(data = df1, columns=['income_type','employment_type','houseown_type','purpose'])

In [55]:
index2 = df_encoding[df_encoding['Month'] == 6].index
df_train = df_encoding.drop(index2)
df_test = df_encoding[df_encoding['Month'] == 6]

In [ ]:
df_train = df_train.dropna(axis=0)

In [57]:
df_train = df_train.drop(['Month'],axis=1)
df_test = df_test.drop(['Month'], axis=1)

# LGBM

In [58]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour

### RobustScaler

In [59]:
X_train = df_train.drop(['is_applied'], axis = 1)
y_train = df_train['is_applied']

In [ ]:
scaler = RobustScaler()
scaler = scaler.fit_transform(X_train)
X_train_1 = pd.DataFrame(scaler)

In [ ]:
lgb_clf = lgb.LGBMClassifier(num_leaves=300, maxdepth = 32 ,objective='binary')
lgb_clf.fit(X_train_1, y_train)

In [ ]:
kfold = KFold(n_splits = 5)
basic_scores = cross_val_score(lgb_clf, X_train, y_train, scoring = 'f1', cv =kfold)

In [ ]:
basic_scores 

## 나이브 베이즈

In [60]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix 

### LabelEncoder

In [61]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [62]:
encoder = MultiColumnLabelEncoder(columns=['income_type','employment_type','purpose','houseown_type'])
df1 = encoder.fit_transform(df1)

In [ ]:
index2 = df_encoding[df_encoding['Month'] == 6].index
df_train = df_encoding.drop(index2)
df_test = df_encoding[df_encoding['Month'] == 6]

In [ ]:
df_train = df_train.dropna(axis=0)

In [ ]:
df_train = df_train.drop(['Month'],axis=1)
df_test = df_test.drop(['Month'], axis=1)

In [67]:
X_train = df_train.drop(['is_applied'], axis = 1)
y_train = df_train['is_applied']

## 다항 나이브 베이즈 분류기 학습

In [68]:
mnb=MultinomialNB()
mnb.fit(X_train,y_train )

ValueError: ignored

In [ ]:
mulitNB = mnb.fit(X_train,y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 5)
f1_score = cross_val_score(mulitNB, X_train,y_train, scoring ='f1', cv = kfold)
f1_score